<a href="https://colab.research.google.com/github/Sakshibisen11/MovieReviews/blob/main/MovieReviews_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2022-09-12 19:38:02--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2022-09-12 19:38:03--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce331475f0afa47e026d70eb451.dl.dropboxusercontent.com/cd/0/inline/Bs0yGjSHx0d0ldzzBdsoAOYIlnf2Hwd_quZXS5oO1V7hYTaKYg1NCxRFJ5N9wYdqMIH6xeO74VpHlQNOcTKsZPRUykUadEvRwoItEA2cKqmY2AArU5rsp01DOExVo8lqVXaIDfp5CPxFnXzSRHeJ3edCoZ58llt-dk15mhf3glIC-A/file# [following]
--2022-09-12 19:38:03--  https://uce331475f0afa47e026d70eb451.dl.dropboxusercontent.com/cd/0/inline/Bs0yGjSHx0d0ldzzBdsoAOYIlnf2Hwd_quZXS5oO1

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense,LSTM,Embedding,Dropout


In [4]:
def prepareData(dir):
  data=text_dataset_from_directory(dir)
  return data.map(
      lambda text,label:(regex_replace(text,'<br/>',''),label),
  )

In [5]:
train_data=prepareData('movie-reviews-dataset/train')
test_data=prepareData('movie-reviews-dataset/test')
for text_batch,label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"Carlos Mencia was excellent this is hour special. He was working hard to show everybody he was the real deal. I know people have said he's stolen material in this special, but that is not true. Carlos brings comedy up front the way he wants it, not how anyone else wants it, that is why he is so good. People say he's not funny because he says Dee dee dee too much, and they still haven't realized thats part of his act, and they don't want it that way, but he brings it like that anyway, and succeeds in making people laugh. For all the haters out there, here is a message, Carlos is here to stay, you have no point in trying to bring him down."
1


In [6]:
model=Sequential()

In [7]:
model.add(Input(shape=(1,),dtype="string"))

In [8]:
#convert string input to vector representation
max_tokens=1000  #tokenize only 1000 words.Rest other words are considered as out of vocabulary
max_len=100 #determines maximum length of the vector
vectorize_layer=TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_len,
)

In [9]:
#extract data from train_data object and train the vectorizor
train_texts=train_data.map(lambda text,label:text)
vectorize_layer.adapt(train_texts)
model.add(vectorize_layer)

In [10]:
model.add(Embedding(max_tokens+1,128))
model.add(LSTM(64))
model.add(Dense(64,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [11]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [12]:
model.fit(train_data,epochs=2)

Epoch 1/2
782/782 [==============================] - 63s 77ms/step - loss: 0.5361 - accuracy: 0.7284
Epoch 2/2
782/782 [==============================] - 57s 73ms/step - loss: 0.4444 - accuracy: 0.7941


In [13]:
model.evaluate(test_data)

782/782 [==============================] - 22s 27ms/step - loss: 0.4478 - accuracy: 0.7868


[0.44779476523399353, 0.7868000268936157]

In [14]:
text="I loved this movie"

In [15]:
model.predict([text]) #positive review

array([[0.93744445]], dtype=float32)